# Exercise Chapter 6 
Industrial Statistics: A Computer Based Approach with Python<br>
by Ron Kenett, Shelemyahu Zacks, Peter Gedeck

Publisher: Springer International Publishing; 1st edition (2023) <br>
<!-- ISBN-13: 978-3031075650 -->

(c) 2022 Ron Kenett, Shelemyahu Zacks, Peter Gedeck

The code needs to be executed in sequence.

In [1]:
import os
os.environ['OUTDATED_IGNORE'] = '1'
import warnings
from outdated import OutdatedPackageWarning
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=OutdatedPackageWarning)

In [2]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf

import mistat

# Exercise 6

In [3]:
df = pd.DataFrame([
  [1, 1, 1, 1, 1, 1, 1, 1, 2.5, 0.0827],
  [1, 1, 2, 2, 2, 2, 2, 2, 2.684, 0.1196],
  [1, 1, 3, 3, 3, 3, 3, 3, 2.66, 0.1722],
  [1, 2, 1, 1, 2, 2, 3, 3, 1.962, 0.1696],
  [1, 2, 2, 2, 3, 3, 1, 1, 1.87, 0.1168],
  [1, 2, 3, 3, 1, 1, 2, 2, 2.584, 0.1106],
  [1, 3, 1, 2, 1, 3, 2, 3, 2.032, 0.0718],
  [1, 3, 2, 3, 2, 1, 3, 1, 3.267, 0.2101],
  [1, 3, 3, 1, 3, 2, 1, 2, 2.829, 0.1516],
  [2, 1, 1, 3, 3, 2, 2, 1, 2.66, 0.1912],
  [2, 1, 2, 1, 1, 3, 3, 2, 3.166, 0.0674],
  [2, 1, 3, 2, 2, 1, 1, 3, 3.323, 0.1274],
  [2, 2, 1, 2, 3, 1, 3, 2, 2.576, 0.085],
  [2, 2, 2, 3, 1, 2, 1, 3, 2.308, 0.0964],
  [2, 2, 3, 1, 2, 3, 2, 1, 2.464, 0.0385],
  [2, 3, 1, 3, 2, 3, 1, 2, 2.667, 0.0706],
  [2, 3, 2, 1, 3, 1, 2, 3, 3.156, 0.1569],
  [2, 3, 3, 2, 1, 2, 3, 1, 3.494, 0.0473],
], columns=['F1', 'F2', 'F3', 'F4', 'F5', 
            'F6', 'F7', 'F8', 'Xbar', 'S'])

In [4]:
df['F1'] = (df['F1']-1)*2-1
for column in ['F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8']:
    df[column] = df[column] - 2
df['SNR'] = np.log(df['Xbar'] / df['S'])

In [5]:
model = smf.ols('SNR ~ F1 + F2 + F3 + F4 + F5 + F6 + F7 + F8', data=df).fit()

In [6]:
# Ignore user warning thrown here
warnings.filterwarnings('ignore', category=UserWarning)

In [7]:
print(model.summary2().tables[1].round(5))

             Coef.  Std.Err.         t    P>|t|   [0.025   0.975]
Intercept  3.22601   0.07518  42.90997  0.00000  3.05594  3.39608
F1         0.26565   0.07518   3.53348  0.00638  0.09558  0.43572
F2         0.07900   0.09208   0.85797  0.41317 -0.12929  0.28729
F3         0.13848   0.09208   1.50392  0.16686 -0.06982  0.34677
F4        -0.14339   0.09208  -1.55733  0.15382 -0.35169  0.06490
F5        -0.31231   0.09208  -3.39185  0.00798 -0.52061 -0.10402
F6         0.12630   0.09208   1.37164  0.20340 -0.08200  0.33459
F7         0.02632   0.09208   0.28587  0.78145 -0.18197  0.23462
F8        -0.17109   0.09208  -1.85810  0.09610 -0.37938  0.03720


In [8]:
warnings.filterwarnings('default', category=UserWarning)